# Creating dynamic flows

In this tutorial, you will:

- Learn how to create dynamic workflows.
- Understand the `detour`, `addition`, and `replace` options in the `Response` object.

The ability create dynamic workflows (i.e. jobs or workflows that launch other jobs or workflows) is
a particularly powerful usage pattern in Jobflow.


## The `Response(replace)` option


The main mechanism for creating dynamic jobs in Jobflow is through the `Response` object. We will demonstrate this below for a toy example where we:

1. Generate a list of numbers whose length is only determined at runtime.
2. Perform a toy operation on each number in the list.

While this is a trivial example, a similar usage is common in computational materials science (e.g. you might perform a calculation on a bulk structure, carve all possible surface slabs, and then perform a calculation on each slab). What makes this dynamic is that the number of jobs is only determined at runtime.


In [1]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

In [2]:
from random import randint
from jobflow import job, Flow, Response
from jobflow.managers.local import run_locally

@job
def make_list(a):
    return [a] * randint(2, 5)

@job
def add(a, b):
    return a + b

@job
def add_distributed(list_a):
    jobs = []
    for val in list_a:
        jobs.append(add(val, 1))
    
    flow = Flow(jobs)
    return Response(replace=flow)

job1 = make_list(2)
job2 = add_distributed(job1.output)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-08 09:55:05,762 INFO Started executing jobs locally
2023-06-08 09:55:05,873 INFO Starting job - make_list (68b72ca9-00ae-430f-a61c-fbea22bbf0aa)
2023-06-08 09:55:05,874 INFO Finished job - make_list (68b72ca9-00ae-430f-a61c-fbea22bbf0aa)
2023-06-08 09:55:05,874 INFO Starting job - add_distributed (01821a62-3ee2-47f0-a6ca-fd373ea0ea09)
2023-06-08 09:55:05,875 INFO Finished job - add_distributed (01821a62-3ee2-47f0-a6ca-fd373ea0ea09)
2023-06-08 09:55:05,876 INFO Starting job - add (fbc49130-2bbe-49b0-a190-48815c03a0b0)
2023-06-08 09:55:05,876 INFO Finished job - add (fbc49130-2bbe-49b0-a190-48815c03a0b0)
2023-06-08 09:55:05,877 INFO Starting job - add (ca3636e3-50f7-4ec8-a5e7-66bc4e14b901)
2023-06-08 09:55:05,877 INFO Finished job - add (ca3636e3-50f7-4ec8-a5e7-66bc4e14b901)
2023-06-08 09:55:05,877 INFO Starting job - add (28dbe7c4-b29b-46c4-9d0c-2aee7334b999)
2023-06-08 09:55:05,878 INFO Finished job - add (28dbe7c4-b29b-46c4-9d0c-2aee7334b999)
2023-06-08 09:55:05,878 INFO Start

/Users/alexmganose/dev/src/jobflow/src/jobflow/utils/graph.py:53: UserWarning: Some jobs are not connected, their ordering may be random
  warnings.warn("Some jobs are not connected, their ordering may be random")


In [3]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

68b72ca9-00ae-430f-a61c-fbea22bbf0aa -> {1: Response(output=[2, 2, 2, 2], detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
01821a62-3ee2-47f0-a6ca-fd373ea0ea09 -> {1: Response(output=None, detour=None, addition=None, replace=<jobflow.core.flow.Flow object at 0x122284cd0>, stored_data=None, stop_children=False, stop_jobflow=False)}
fbc49130-2bbe-49b0-a190-48815c03a0b0 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
ca3636e3-50f7-4ec8-a5e7-66bc4e14b901 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
28dbe7c4-b29b-46c4-9d0c-2aee7334b999 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
1bffefd1-4edc-43d5-b99f-b0833fca2b8d -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=No

As seen above, there are several jobs that were run --- certainly more than the two we started with. The first job generates a list of 2's with a random length. The second job in the flow is what launches a job on each entry in the list. It is replaced by one job for each entry, hence it has no direct output. Then each newly generated job is run.


## The `Response(addition)` option


Beyond replacing a job with downstream jobs, there is also the option to add jobs to the current flow on-the-fly.

Here we will create a simple flow that:

1. Adds a value to a given number.
2. If the output is less than 10, do the addition again. Otherwise, stop.


In [4]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-08 09:55:07,374 INFO Started executing jobs locally
2023-06-08 09:55:07,377 INFO Starting job - add (8270f7cc-e12a-457b-adca-a196ee62ae85)
2023-06-08 09:55:07,379 INFO Finished job - add (8270f7cc-e12a-457b-adca-a196ee62ae85)
2023-06-08 09:55:07,380 INFO Starting job - add_with_logic (fd549146-aabb-4e4b-b79a-ca6b6fd2c172)
2023-06-08 09:55:07,382 INFO Finished job - add_with_logic (fd549146-aabb-4e4b-b79a-ca6b6fd2c172)
2023-06-08 09:55:07,384 INFO Starting job - add (9db2454f-4f1d-4d08-8497-1239cad74d0c)
2023-06-08 09:55:07,384 INFO Finished job - add (9db2454f-4f1d-4d08-8497-1239cad74d0c)
2023-06-08 09:55:07,385 INFO Finished executing jobs locally


In [5]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

8270f7cc-e12a-457b-adca-a196ee62ae85 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
fd549146-aabb-4e4b-b79a-ca6b6fd2c172 -> {1: Response(output=None, detour=None, addition=<jobflow.core.flow.Flow object at 0x12229b0d0>, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
9db2454f-4f1d-4d08-8497-1239cad74d0c -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


As you can see above, the addition job is correctly run twice. Now let's confirm that the addition job is only run once if the output of the first job is greater than 10.


In [6]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 20)
job2 = add_with_logic(job1.output, 20)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-08 09:55:08,838 INFO Started executing jobs locally
2023-06-08 09:55:08,841 INFO Starting job - add (ec551667-1500-45cc-b61a-2361a18b99fd)
2023-06-08 09:55:08,843 INFO Finished job - add (ec551667-1500-45cc-b61a-2361a18b99fd)
2023-06-08 09:55:08,843 INFO Starting job - add_with_logic (e7d3fb53-fa56-45bd-9951-45b70028ada7)
2023-06-08 09:55:08,846 INFO Finished job - add_with_logic (e7d3fb53-fa56-45bd-9951-45b70028ada7)
2023-06-08 09:55:08,847 INFO Finished executing jobs locally


In [7]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

ec551667-1500-45cc-b61a-2361a18b99fd -> {1: Response(output=21, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
e7d3fb53-fa56-45bd-9951-45b70028ada7 -> {1: Response(output=None, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Now, we see that the `Response(addition)` does not launch a new job.


## The `Response(detour)` option


The `Response(detour)` option behaves similarly to `Response(addition)`. The difference is that `Response(addition)` will add a job (or flow) to the current flow, while `Response(detour)` will no longer run the current flow and will switch to a parallel job or flow.


In [8]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(detour=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-08 09:55:10,850 INFO Started executing jobs locally
2023-06-08 09:55:10,853 INFO Starting job - add (d5020d71-4bbc-4307-8fa5-c65d0d9b9b9a)
2023-06-08 09:55:10,855 INFO Finished job - add (d5020d71-4bbc-4307-8fa5-c65d0d9b9b9a)
2023-06-08 09:55:10,856 INFO Starting job - add_with_logic (8786b3c4-5697-4ed8-9f46-fd0c89e4c374)
2023-06-08 09:55:10,858 INFO Finished job - add_with_logic (8786b3c4-5697-4ed8-9f46-fd0c89e4c374)
2023-06-08 09:55:10,859 INFO Starting job - add (c33d72c2-f396-4f2c-898e-c667c0913dcb)
2023-06-08 09:55:10,859 INFO Finished job - add (c33d72c2-f396-4f2c-898e-c667c0913dcb)
2023-06-08 09:55:10,860 INFO Finished executing jobs locally


In [9]:
responses

{'d5020d71-4bbc-4307-8fa5-c65d0d9b9b9a': {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)},
 '8786b3c4-5697-4ed8-9f46-fd0c89e4c374': {1: Response(output=None, detour=<jobflow.core.flow.Flow object at 0x12229bc70>, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)},
 'c33d72c2-f396-4f2c-898e-c667c0913dcb': {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}}

For this toy example, both `Response(addition)` and `Response(detour)` behave identically.
